# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from collections import defaultdict
from fastcore.all import *
from inspect import Parameter, Signature

import httpx, re

In [ ]:
#| hide
import os

In [ ]:
#| export
stripe_openapi_url = 'https://raw.githubusercontent.com/stripe/openapi/refs/heads/master/openapi/spec3.json'
stripe_spec = httpx.get(stripe_openapi_url).json()

In [ ]:
stripe_spec.keys()

dict_keys(['components', 'info', 'openapi', 'paths', 'security', 'servers'])

The OpenAPI spec describes how a particular REST API works. The most important part of this spec is the paths that are defined by a particular spec. This defines what you can do with a particular API and usually includes things like the description of endpoint, expected parameters, the description of the parameters, and a schema of what will be returned from a particular endpoint.

In [ ]:
p = first(stripe_spec['paths'].items())
p

('/v1/account',
 {'get': {'description': '<p>Retrieves the details of an account.</p>',
   'operationId': 'GetAccount',
   'parameters': [{'description': 'Specifies which fields in the response should be expanded.',
     'explode': True,
     'in': 'query',
     'name': 'expand',
     'required': False,
     'schema': {'items': {'maxLength': 5000, 'type': 'string'},
      'type': 'array'},
     'style': 'deepObject'}],
   'requestBody': {'content': {'application/x-www-form-urlencoded': {'encoding': {},
      'schema': {'additionalProperties': False,
       'properties': {},
       'type': 'object'}}},
    'required': False},
   'responses': {'200': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/account'}}},
     'description': 'Successful response.'},
    'default': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/error'}}},
     'description': 'Error response.'}},
   'summary': 'Retrieve account'}})

As we can see here, the account path has a single HTTP verb that we can use on it called GET. The verb + path is what we refer to as an endpoint. This endpoint allows us to get the account details for a stripe.

In [ ]:
#| export
stripe_api_url = 'https://api.stripe.com'

In [ ]:
stripe_api_url + p[0]

'https://api.stripe.com/v1/account'

In [ ]:
stripe_key = os.getenv('STRIPE_SECRET_KEY')
headers = {'Authorization': f'Bearer {stripe_key}'}
resp = httpx.get(stripe_api_url + p[0], headers=headers)
# resp.status_code, resp.json()

Some of these endpoints will take parameters if they are GET verbs or request bodies. POST verbs. Here is an example for the GET customers endpoint

In [ ]:
p = first(stripe_spec['paths'].items(), lambda x: x[0] == '/v1/customers')
p[1]['get']['parameters'][:3]

[{'description': 'Only return customers that were created during the given date interval.',
  'explode': True,
  'in': 'query',
  'name': 'created',
  'required': False,
  'schema': {'anyOf': [{'properties': {'gt': {'type': 'integer'},
      'gte': {'type': 'integer'},
      'lt': {'type': 'integer'},
      'lte': {'type': 'integer'}},
     'title': 'range_query_specs',
     'type': 'object'},
    {'type': 'integer'}]},
  'style': 'deepObject'},
 {'description': "A case-sensitive filter on the list based on the customer's `email` field. The value must be a string.",
  'in': 'query',
  'name': 'email',
  'required': False,
  'schema': {'maxLength': 512, 'type': 'string'},
  'style': 'form'},
 {'description': 'A cursor for use in pagination. `ending_before` is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, starting with `obj_bar`, your subsequent call can include `ending_before=obj_bar` in order to fetch the previous pa

In [ ]:
list(p[1]['post']['requestBody']['content']['application/x-www-form-urlencoded']['schema']['properties'].items())[:3]

[('address',
  {'anyOf': [{'properties': {'city': {'maxLength': 5000, 'type': 'string'},
      'country': {'maxLength': 5000, 'type': 'string'},
      'line1': {'maxLength': 5000, 'type': 'string'},
      'line2': {'maxLength': 5000, 'type': 'string'},
      'postal_code': {'maxLength': 5000, 'type': 'string'},
      'state': {'maxLength': 5000, 'type': 'string'}},
     'title': 'optional_fields_customer_address',
     'type': 'object'},
    {'enum': [''], 'type': 'string'}],
   'description': "The customer's address."}),
 ('balance',
  {'description': "An integer amount in cents (or local equivalent) that represents the customer's current balance, which affect the customer's future invoices. A negative amount represents a credit that decreases the amount due on an invoice; a positive amount increases the amount due on an invoice.",
   'type': 'integer'}),
 ('cash_balance',
  {'description': 'Balance information and default balance settings for this customer.',
   'properties': {'setti

Let's make a helper function to grab all these endpoints and their parameters.

In [ ]:
#| export
def stripe_endpoints(spec: dict):
    'Extracts all the endpoints and their parameters from the Stripe OpenAPI spec.'
    endpoints = []
    for path, methods in spec['paths'].items():
        for verb, details in methods.items():
            op_id = details.get('operationId', '')
            summary = details.get('summary', '')
            query_params = [dict(name=p['name'], description=p.get('description', ''))
                            for p in details.get('parameters', []) if p.get('in') == 'query']
            body_params = []
            if 'requestBody' in details:
                schema = nested_idx(details, 'requestBody', 'content', 'application/x-www-form-urlencoded', 'schema', 'properties') or {}
                body_params = [dict(name=k, description=v.get('description', '')) for k,v in schema.items()]
            all_params = query_params + body_params
            endpoints.append(dict(path=path, verb=verb, op_id=op_id, summary=summary, params=all_params))
    return endpoints


In [ ]:
#| export
eps = stripe_endpoints(stripe_spec)

In [ ]:
eps[:3]

[{'path': '/v1/account',
  'verb': 'get',
  'op_id': 'GetAccount',
  'summary': 'Retrieve account',
  'params': [{'name': 'expand',
    'description': 'Specifies which fields in the response should be expanded.'}]},
 {'path': '/v1/account_links',
  'verb': 'post',
  'op_id': 'PostAccountLinks',
  'summary': 'Create an account link',
  'params': [{'name': 'account',
    'description': 'The identifier of the account to create an account link for.'},
   {'name': 'collect',
    'description': 'The collect parameter is deprecated. Use `collection_options` instead.'},
   {'name': 'collection_options',
    'description': 'Specifies the requirements that Stripe collects from connected accounts in the Connect Onboarding flow.'},
   {'name': 'expand',
    'description': 'Specifies which fields in the response should be expanded.'},
   {'name': 'refresh_url',
    'description': "The URL the user will be redirected to if the account link is expired, has been previously-visited, or is otherwise inv

Now, with each of these descriptions, we can easily create a request that we want on the fly. However, to make it a little bit nicer to use in a library, we'll go ahead and automatically generate functions with proper signatures and docstrings that are then easily accessible in any standard IDE.

In [ ]:
#| export
def _flatten_data(data, prefix=''):
    'Flatten a dictionary of data so that it can be used in a request body.'
    result = {}
    for k,v in data.items():
        key = f'{prefix}[{k}]' if prefix else k
        if isinstance(v, dict): result.update(_flatten_data(v, key))
        elif isinstance(v, list): 
            for i,item in enumerate(v):
                if isinstance(item, dict): result.update(_flatten_data(item, f'{key}[{i}]'))
                else: result[f'{key}[{i}]'] = item
        else: result[key] = v
    return result

In [ ]:
def _mk_func(path, verb, param_info, summary):
    sig_params = [Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) for param in param_info]
    param_docs = '\n'.join(f"    {param['name']}: {param['description']}" for param in param_info)
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    def m(**kwargs): return getattr(httpx, verb)(stripe_api_url + path, headers=headers, params=_flatten_data(kwargs)).json()
    m.__signature__ = Signature(sig_params)
    m.__doc__ = docstring
    return m

In [ ]:
acc_ep = eps[0]
get_account = _mk_func(acc_ep['path'], acc_ep['verb'], acc_ep['params'], acc_ep['summary'])
get_account??

Signature: get_account(*, expand=None)
Docstring:
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
Source:        def m(**kwargs): return getattr(httpx, verb)(stripe_api_url + path, headers=headers, params=_flatten_data(kwargs)).json()
File:      /var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_32957/3787833087.py
Type:      function

In [ ]:
# get_account()

Let's go ahead and design a class since we need to store our API key and for use in the headers for each of these functions. 

In [ ]:
#| export
def _parse_operation_id(op_id):
    parts = re.findall(r'[A-Z][a-z]*', op_id)
    verb,*resource_parts = [p.lower() for p in parts]
    r = '_'.join(resource_parts) if resource_parts else 'misc'
    nm = 'create' if verb == 'post' else 'fetch' if verb == 'get' else verb
    return r, nm

For the name of our function, we're going to use this operation ID. However, they use title case so let's make a helper to properly split and construct these names

In [ ]:
r, nm = _parse_operation_id(acc_ep['op_id'])
f'{r}.{nm}'

'account.fetch'

**Note:** We can't set these as the same post/get verbs because when we actually add these as attributes in our class, we're going to use an AttrDict which we cannot overwrite the get method. So I went ahead and changed the names to either fetch or create. 

In [ ]:
#| export
def _mk_func(path, verb, param_info, summary, hdrs={}):
    sig_params = [Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) for param in param_info]
    param_docs = '\n'.join(f"    {param['name']}: {param['description']}" for param in param_info)
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    def m(**kwargs): return dict2obj(getattr(httpx, verb)(stripe_api_url + path, headers=hdrs,
                                                          params=_flatten_data(kwargs)).json())
    m.__signature__ = Signature(sig_params)
    m.__doc__ = docstring
    return m

In [ ]:
#| export
class StripeApi:
    def __init__(self, api_key=None, base_url='https://api.stripe.com'):
        self.api_key,self.base_url = api_key,base_url
        self.hdrs = {'Authorization': f'Bearer {self.api_key}'}
        eps = stripe_endpoints(stripe_spec)
        groups = defaultdict(list)
        for ep in eps:
            r, nm = _parse_operation_id(ep['op_id'])
            groups[r].append((ep['path'], ep['verb'], nm, ep['summary'], ep['params']))
        for r, mtds in groups.items():
            group = AttrDict()
            for path, verb, method_name, summary, params in mtds:
                setattr(group, method_name, _mk_func(path, verb, params, summary, hdrs=self.hdrs))
            setattr(self, r, group)

In [ ]:
sapi = StripeApi(api_key=stripe_key)
sapi.account.fetch?

Signature: sapi.account.fetch(*, expand=None)
Docstring:
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
File:      /var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_32957/959316412.py
Type:      function

That is all we need in order to have a fully functional Python SDK that is compliant with the Stripe OpenAPI spec. Kind of insane that in under 100 lines of code, we can get this functionality, which in my opinion is in some respects even better than the official Stripe Python SDK for the simple fact that we can see the parameters that the functions take without looking up the API reference doc online.

Let's go ahead and try to build with this thing. The simplest payment system that you can have in Stripe is a one-time payment URL. Here is how we do this in our new API. First, we have to create a product and its price.

In [ ]:
prod = sapi.products.create(name='Test Product')
prod.id, prod.name

('prod_SMsKL8PazjzxCp', 'Test Product')

In [ ]:
price = sapi.prices.create(product=prod.id, unit_amount=1000, currency='usd')
price.id, price.unit_amount, price.currency

('price_1RS8a7KGhqIw9PXmCy76wa1M', 1000, 'usd')

Now we can create our checkout session with a mode of payment which means that it will only happen once and is not part of any sort of subscription.

In [ ]:
checkout = sapi.checkout_sessions.create(mode='payment', line_items=[dict(price=price.id, quantity=1)],
                                         success_url='https://localhost:8000/success', cancel_url='https://localhost:8000/cancel')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1GI49RQWTH9v12ndyoCgNKW...


Let's make this process even easier by add a higher level api ontop of our StripeApi.

First, let's make it a little bit easier to find an existing products and prices.

In [ ]:
#| export
@patch
def find_product(self:StripeApi, name: str):
    "Find a product by name"
    prods = L(self.products.fetch().data)
    return first(prods, lambda p: p.name == name)

In [ ]:
sapi.find_product('Test Product')

```json
{ 'active': True,
  'attributes': [],
  'created': 1748056627,
  'default_price': None,
  'description': None,
  'id': 'prod_SMsKL8PazjzxCp',
  'images': [],
  'livemode': False,
  'marketing_features': [],
  'metadata': {},
  'name': 'Test Product',
  'object': 'product',
  'package_dimensions': None,
  'shippable': None,
  'statement_descriptor': None,
  'tax_code': None,
  'type': 'service',
  'unit_label': None,
  'updated': 1748056627,
  'url': None}
```

In [ ]:
#| export
@patch
def find_prices(self:StripeApi, product_id: str):
    "Find all prices associated with a product id"
    return L(self.prices.fetch().data).filter(lambda p: p.product == product_id)

In [ ]:
sapi.find_prices(sapi.find_product('Test Product').id)

(#1) [{'id': 'price_1RS8a7KGhqIw9PXmCy76wa1M', 'object': 'price', 'active': True, 'billing_scheme': 'per_unit', 'created': 1748056627, 'currency': 'usd', 'custom_unit_amount': None, 'livemode': False, 'lookup_key': None, 'metadata': {}, 'nickname': None, 'product': 'prod_SMsKL8PazjzxCp', 'recurring': None, 'tax_behavior': 'unspecified', 'tiers_mode': None, 'transform_quantity': None, 'type': 'one_time', 'unit_amount': 1000, 'unit_amount_decimal': '1000'}]

In [ ]:
#| export
@patch
def priced_product(self:StripeApi, product_name, amount_cents, currency='usd', recurring=None):
    "Create a product and price if they don't exist"
    prod = self.find_product(product_name) or self.products.create(name=product_name)
    params = dict(product=prod.id, unit_amount=amount_cents, currency=currency)
    if recurring: params['recurring'] = recurring
    price = first(self.find_prices(prod.id)) or self.prices.create(**params)
    return prod, price

In [ ]:
prod, price = sapi.priced_product('Test Product', 1000, 'usd')

Now we can automatically create a product or use an existing one when create a one time payment link.

In [ ]:
#| export
@patch
def one_time_payment(self:StripeApi, product_name, amount_cents, success_url, cancel_url, currency='usd'):
    "Create a simple one-time payment checkout"
    _, price = self.priced_product(product_name, amount_cents, currency)
    return self.checkout_sessions.create(mode='payment', line_items=[dict(price=price.id, quantity=1)],
                                         success_url=success_url, cancel_url=cancel_url)

In [ ]:
checkout = sapi.one_time_payment('Test Product', 1000, 'https://localhost:8000/success', 'https://localhost:8000/cancel', 'usd')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1tG6yGlb6sv8tRK6nOHJ1D9...


Another common use case is subscriptions. Let's go ahead and create a version that makes subscription creation just as easy.

In [ ]:
#| export
@patch
def subscription(self:StripeApi, product_name, amount_cents , success_url, cancel_url,
                 currency='usd', interval='month', customer_email=None):
    "Create a simple recurring subscription"
    _, price = self.priced_product(product_name, amount_cents, currency, recurring=dict(interval=interval))
    
    params = dict(mode='subscription', line_items=[dict(price=price.id, quantity=1)],
                  success_url=success_url, cancel_url=cancel_url)
    if customer_email: params['customer_email'] = customer_email
    return self.checkout_sessions.create(**params)

In [ ]:
sub_checkout = sapi.subscription('Test Subscription Product', 1000, 'https://localhost:8000/success', 'https://localhost:8000/cancel')
print(f'Payment link: {sub_checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1SXP3jSsh0W5nrpX9XRAP5L...


**Note:** You'll want to use Stripe's webhook functionality for detecting payment and subscription events. To do so, you'll utilize the Python Stripe SDK that the event actually came from stripe.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()